In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing.whatifs import scale_up_acc_to_testing

from models.seir import SEIRHD, SEIR_Testing, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.uncertainty import MCUncertainty
from main.seir.optimiser import Optimiser
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv, get_all_trials
from main.seir.sensitivity import gridsearch_single_param, calculate_sensitivity_and_plot, plot_single_comp

from utils.create_report import create_report, trials_to_df
from utils.loss import Loss_Calculator
from utils.enums import Columns
from utils.enums.columns import *

from viz import plot_forecast, plot_trials, plot_fit, plot_fit_multiple_preds

## Load Covid19india Data

In [ ]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
districts_to_show = [('Maharashtra', 'Mumbai')]

## Params

In [ ]:
date_of_interest = '2020-07-31'
forecast_days = 37
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)
ktrials = 10

## Perform M1 and M2 fits

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=False, granular_data=False, filename=None, #Data
        model=SEIRHD, #Choose Model and Ranges
        train_period=21, val_period=7, back_offset=0, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased'], #Compartments to Apply Loss on 
        smooth_jump=True)
    
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=False, granular_data=False, filename=None, #Data
        model=SEIRHD, #Choose Model and Ranges
        train_period=21, val_period=0, back_offset=0, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased'], #Compartments to Apply Loss on 
        smooth_jump=True)
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']
    predictions_dict[(state, district)]['data_last_date'] = predictions_dict[(state, district)]['m2']['data_last_date']

## Loss DataFrames

### M1 Loss DataFrame

In [ ]:
lc = Loss_Calculator()

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### M2 Loss DataFrame

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['m2']['df_forecast'] = get_forecast(predictions_dict[region], days=forecast_days)
    predictions_dict[region]['m2']['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=False, error_bars=True, days=forecast_days)
    
    predictions, losses, params = get_all_trials(predictions_dict[region], train_fit='m1')
    predictions_dict[region]['m1']['params'] = params
    predictions_dict[region]['m1']['losses'] = losses
    predictions_dict[region]['m1']['predictions'] = predictions
    predictions_dict[region]['m1']['all_trials'] = trials_to_df(predictions, losses, params)
    predictions, losses, params = get_all_trials(predictions_dict[region], train_fit='m2')
    predictions_dict[region]['m2']['params'] = params
    predictions_dict[region]['m2']['losses'] = losses
    predictions_dict[region]['m2']['predictions'] = predictions
    predictions_dict[region]['m2']['all_trials'] = trials_to_df(predictions, losses, params)
    kforecasts = plot_trials(
        predictions_dict[region],
        train_fit='m2',
        predictions=predictions, 
        losses=losses, params=params, 
        k=ktrials,
        which_compartments=[Columns.confirmed, Columns.active])
    predictions_dict[region]['m2']['forecast_confirmed_topk'] = kforecasts[Columns.confirmed]
    predictions_dict[region]['m2']['forecast_active_topk'] = kforecasts[Columns.active]

## Uncertainty

In [ ]:
region_dict = predictions_dict[('Maharashtra', 'Mumbai')]
uncertainty = MCUncertainty(region_dict, date_of_interest)

In [ ]:
uncertainty.beta

## Whatifs

In [ ]:
region_dict = predictions_dict[('Maharashtra', 'Mumbai')]
which_fit = 'm2'
time_window_to_scale = 14

In [ ]:
def scale_up_testing_and_forecast(testing_scaling_factor=1.5):
    df_whatif = scale_up_acc_to_testing(region_dict, testing_scaling_factor=testing_scaling_factor, time_window_to_scale=time_window_to_scale)

    optimiser = Optimiser()
    extra_params = optimiser.init_default_params(df_whatif, N=1e7, initialisation='intermediate', train_period=time_window_to_scale)
    best_params = copy.copy(region_dict[which_fit]['best_params'])
    del best_params['T_inf']
    del best_params['E_hosp_ratio']
    del best_params['I_hosp_ratio']
    default_params = {**extra_params, **best_params}

    total_days = (df_whatif.iloc[-1, :]['date'] - default_params['starting_date']).days
    variable_param_ranges = {
        'T_inf': (0.01, 10),
        'E_hosp_ratio': (0, 2),
        'I_hosp_ratio': (0, 1)
    }
    variable_param_ranges = get_variable_param_ranges(variable_param_ranges=variable_param_ranges)
    best, trials = optimiser.bayes_opt(df_whatif, default_params, variable_param_ranges, model=SEIR_Testing, total_days=total_days, 
                                       method='mape', num_evals=500, loss_indices=[-time_window_to_scale, None], 
                                       which_compartments=['total_infected'])

    df_prediction = optimiser.solve(best, default_params, 
                                    region_dict[which_fit]['df_train'], 
                                    end_date=region_dict[which_fit]['df_forecast'].iloc[-1, :]['date'], 
                                    model=region_dict[which_fit]['run_params']['model_class'])
    return df_prediction

In [ ]:
df_prediction_15 = scale_up_testing_and_forecast(testing_scaling_factor=1.5)
df_prediction_15

In [ ]:
df_prediction_12 = scale_up_testing_and_forecast(testing_scaling_factor=1.2)
df_prediction_12

In [ ]:
df_prediction_20 = scale_up_testing_and_forecast(testing_scaling_factor=2.0)
df_prediction_20

In [ ]:
df_prediction_25 = scale_up_testing_and_forecast(testing_scaling_factor=2.5)
df_prediction_25

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
for compartment in compartments['base']:
    ax.plot(region_dict['m1']['df_district'][compartments['date'][0].name], region_dict['m1']['df_district'][compartment.name],
            '-o', color=compartment.color, label='{} (Observed)'.format(compartment.label))
    ax.plot(region_dict[which_fit]['df_forecast'][compartments['date'][0].name], region_dict[which_fit]['df_forecast'][compartment.name],
            '-', color=compartment.color, label='{} (Std Forecast)'.format(compartment.label))
    ax.plot(df_prediction_12[compartments['date'][0].name], df_prediction_12[compartment.name],
            '--', color=compartment.color, label='{} (Testing becomes 1.2x)'.format(compartment.label))
    ax.plot(df_prediction_15[compartments['date'][0].name], df_prediction_15[compartment.name],
            ':', color=compartment.color, label='{} (Testing becomes 1.5x)'.format(compartment.label))
    ax.plot(df_prediction_20[compartments['date'][0].name], df_prediction_20[compartment.name],
            '-.', color=compartment.color, label='{} (Testing becomes 2x)'.format(compartment.label))
#     ax.plot(df_prediction_25[compartments['date'][0].name], df_prediction_25[compartment.name],
#             '-x', color=compartment.color, label='{} (Testing becomes 2.5x)'.format(compartment.label))
        
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.ylabel('No of People', fontsize=16)
plt.xlabel('Time', fontsize=16)
plt.xticks(rotation=45, horizontalalignment='right')
plt.legend()
plt.title('Forecast - ({} {})'.format(region[0], region[1]), fontsize=16)
plt.grid()

## Create Report


In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region], ROOT_DIR=output_folder)
    predictions_dict[region]['m1']['all_trials'].to_csv(os.path.join(output_folder, 'm1-trials.csv'))
    predictions_dict[region]['m2']['all_trials'].to_csv(os.path.join(output_folder, 'm2-trials.csv'))

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output, filename=os.path.join(output_folder, f'output-{t}.csv'))